In [1]:
#!pip install openai
import os
import pdb
import time
import openai
import requests
import pandas as pd 

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score, f1_score

In [4]:
# change this key frequently
OPENAI_API_KEY = ""

In [5]:
openai.api_key = OPENAI_API_KEY

In [6]:
def label_message(meta_statement, definition, article, prompt, examples, role_user="user"):

    completion = openai.chat.completions.create(model="gpt-4",
                                              temperature = 0, # make sure that all answers are the same, 
                                              max_tokens = 50, # 8,000 is the max token length for GPT3.5
                                              messages=[{"role": "system", "content": "You are a news classifier."},
                                                        {"role": role_user, "content": meta_statement},
                                                        #{"role": role_user, "content": f'Read this definition {definition}'},
                                                        {"role": role_user, "content": f'Read this article {article}'},
                                                        {"role": role_user, "content": prompt},
                                                        {"role": role_user, "content": "Here are a few examples :\n" + examples}])
                                                        #{"role": role_user, "content": "Input :" + article}, 
                                                        #{"role": role_user, "content": "Output: "}]) ## this is to tell gpt 3.5 to give output according to structure 

    return completion.choices[0].message

In [7]:
news_articles = pd.read_json("old_data_clean.json")

In [9]:
def map_values(value):
    if value == 'irrelevant':
        return '5'
    elif value == 'relchanges':
        return '3'
    elif value == 'explicit':
        return '1'
    elif value == 'releconomy':
        return '4'
    elif value == 'implicit':
        return '2'
    else:
        return 'Unknown'

In [10]:
# Apply the function to create the new column
news_articles['label'] = news_articles['five_code'].apply(map_values)
news_articles['label'].unique()

array(['5', '3', '1', '4', '2'], dtype=object)

In [11]:
rel_news_articles = news_articles[news_articles['label'] != '5'].reset_index(drop=True)
len(rel_news_articles)

786

In [13]:
inequality_def = """Criteria for referencing economic inequality in the United States:
(1) Wage, earnings, pay, and income inequality. 
(2) Causes and policy solutions related to economic inequality. 
(3) Relational or comparative language among different social class groups."""

In [14]:
research_goal = """We categorize articles that are related to issues of income inequality, changes in income or wealth, general economic conditions."""

In [15]:
inequality_def

'Criteria for referencing economic inequality in the United States:\n(1) Wage, earnings, pay, and income inequality. \n(2) Causes and policy solutions related to economic inequality. \n(3) Relational or comparative language among different social class groups.'

In [16]:
research_goal

'We categorize articles that are related to issues of income inequality, changes in income or wealth, general economic conditions.'

In [17]:
example_articles = """Input: Blame Game An unalterable truth in Washington is that everything is about the next election--as President Bush is about to find out in the upcoming battle of the budget. In the Clinton era, budget wars were rollicking political brawls with defining consequences. The best and most obvious example: the government shutdown of 1995. With the economy lagging, the surplus evaporating, and the Democrats controlling the Senate, President Bush enters his first budget fight with fewer advantages than he held on Inauguration Day. Soon he must confront the reality that while the budget deals with the nitty-gritty of collecting and spending some $ 2 trillion annually, it also provides a grand stage for unsubtle overtures to voters, who go to the polls next year to elect all 435 members of the House and more than a third of the Senate. With three prominent GOP senators announcing their retirement from the closely divided Senate, Republicans have 20 seats to defend, while Democrats have only 14 incumbents up for re-election. With the planned departures of Strom Thurmond, Jesse Helms, and Phil Gramm, the Republican task of retaking the Senate grows more daunting by the day. A Democrat-controlled Senate in 2002 means that President Bush will go into his own re-election two years later with South Dakota Democrat Tom Daschle as majority leader, one who could derail the White House's agenda. Reveling in a sooner than expected bout of I-told-you-so, Democrats point to the dwindling federal budget surplus and warn that the Bush administration will be forced to break its campaign pledge that it would not use Social Security taxes for anything other than Social Security benefits. Daschle has been adroitly laying the foundation for blame. "Certainly, they are the architects of our current economic condition, and I think we have to turn to them for leadership now as we find our way out of this box that has been created," he says. The pressure points are obvious. Bush has requested an additional $ 18.3 billion for defense. There has been little progress on how to fund the president's educational reform package. His energy plan is unresolved. On the other side, Democrats want to increase the minimum wage, which most in the GOP oppose if there aren't tax breaks for small businesses. And the Democrats want a prescription drug benefit for Medicare recipients, which could cost $ 300 billion over the next decade. Bush defends his budget by saying that it could accommodate his cherished tax cut and other proposals if only Congress could restrain its chronic tendencies toward spending. But those wise to Washington's ways also know that spending has its virtues. After all, many of the millions spent on so-called pet projects will figure prominently in the political stump speeches during the next election cycle. Output: Yes.
              Input: State and local tax collections reached $320.2 billion in 1984, an increase of 12.5 percent over the previous year, the Commerce Department reported. The state-and-local tax bite averaged $1,356 per person -- or $117 for each $1,000 of personal income. Alaska led the nation with $4,704 per person and $286 per $1,000 of income. Per Per $1,000 capita income Alaska $4,704 $286 Wyoming $2,504 $209 Dist. of Columbia $2,300 $146 New York $2,130 $165 Minnesota $1,706 $144 Connecticut $1,656 $112 New Jersey $1,637 $117 Michigan $1,575 $138 Wisconsin $1,556 $137 Massachusetts $1,549 $117 Hawaii $1,543 $129 Maryland $1,503 $117 California $1,503 $115 Washington $1,416 $118 Illinois $1,405 $113 Rhode Island $1,403 $121 Delaware $1,400 $112 U.S. average $1,356 $117 Nevada $1,353 $111 Colorado $1,339 $106 North Dakota $1,334 $115 Oregon $1,321 $124 Pennsylvania $1,309 $114 Montana $1,275 $129 Iowa $1,273 $119 Vermont $1,271 $129 Kansas $1,260 $103 Arizona $1,246 $120 Ohio $1,246 $111 Nebraska $1,232 $110 Maine $1,229 $126 Virginia $1,210 $101 New Mexico $1,194 $126 Oklahoma $1,159 $106 Utah $1,133 $129 Texas $1,115 $ 97 Louisiana $1,114 $109 West Virginia $1,113 $121 Indiana $1,093 $105 New Hampshire $1,092 $ 93 Georgia $1,073 $105 Florida $1,073 $ 95 North Carolina $1,027 $106 Missouri $1,012 $ 93 South Carolina $ 981 $108 South Dakota $ 978 $100 Kentucky $ 955 $102 Idaho $ 953 $101 Alabama $ 916 $100 Tennessee $ 878 $ 93 Mississippi $ 871 $108 Arkansas $ 866 $ 97. Output: No."""

In [19]:
cols = rel_news_articles.columns.to_list()

In [20]:
keep_cols = cols[:37] 
keep_cols.append('token_count')
#keep_cols

In [21]:
# only binary 1: 
binary1 = rel_news_articles[keep_cols]

In [ ]:
messages = []
max_char = 4000*3 
#max_token = 7500 # cushion for definition and meta-statement
for i in range(len(binary1)):
    if i % 100 == 0: 
        print("processing message", i)
    title = binary1.loc[i, 'title']
    content = binary1.loc[i, 'text']
    #if binary1.loc[i,'token_count'] <= max_token: 
    #    content = f'title: {title} \n article: {content}'
    #else: 
    #    content = f'title: {title} \n article: {content[:max_char]}'
    content = f'title: {title} \n article: {content[:max_char]}'
    article_prompt = "Does the article reference American economic inequality? Respond with 'Yes' if article meets any or all criteria referencing American economic inequality and 'No' if article meets none of the criteria, and explain why in 1 sentence."
    while True: 
        try: 
            output_message = label_message(meta_statement=research_goal, 
                                           article=content, 
                                           definition=inequality_def, 
                                           examples=example_articles,
                                           prompt=article_prompt)
            break 
        except Exception as e:
             time.sleep(30)       # add time lag of 30 seconds when an error occurs 
    messages.append(output_message) 
#     pdb.set_trace()

In [25]:
binary1['output_message'] = [message.content for message in messages]

In [26]:
binary1['output_message'] = binary1['output_message'].str.lower()

In [28]:
def parse_answers(binary1, messages): 
    '''
    parse the answer to binary no - non-inequality, yes - inequality 
    '''
    predicted = []
    for message in messages: 
        if message.startswith("yes"): 
            predicted.append('yes')
        elif message.startswith("no"): 
            predicted.append('no')
        else: 
            predicted.append('flag')
    binary1['predicted_label'] = predicted
    return 

In [29]:
parse_answers(binary1, binary1['output_message'])

In [45]:
# save the output for future comparison if needed 
binary1.to_csv("../data/inequality_dataset_GPT_Inequality-FewShot-NoDefinition.csv", index=False)